In [1]:
import email
import os
import re
import hashlib
import quopri

In [2]:
INPUT_DIR = '/Users/worshamn/Documents/emailProject/Ignore'
OUTPUT_DIR = '/Users/worshamn/Documents/emailProject/IgnoreFix'

In [3]:
def find_message_attachment(mail):
    if mail.is_multipart():
        for i in mail.get_payload():
            try:
                if i.get_content_maintype() == 'message':
                    return i.get_payload()[0]
            except:
                continue

In [4]:
#https://github.com/ActiveState/code/blob/master/recipes/Python/302086_Strip_attachments_email/recipe-302086.py
#https://github.com/xme/mime2vt/blob/master/mime2vt.py
#https://stackoverflow.com/a/370038
def strip_attachments(message):
    if not message.is_multipart():
        reformatted_message = quopri.decodestring(
                                message.as_string().encode('ascii', 'ignore')
                            ).decode("utf-8",'ignore')
        return reformatted_message
    boundary = message.get_boundary()
    new_payload = '--' + boundary
    for i in message.get_payload():
        if i.get_content_maintype() == 'message':
            return strip_attachements(i)
        elif i.get_content_maintype() == 'text' \
                or i.get_content_maintype() == 'multipart':
            text_content = i.as_string().encode('ascii', 'ignore')
            text_content = quopri.decodestring(text_content).decode("utf-8",'ignore')
            new_payload += '\n' + text_content
        else:
            replace = re.sub(r'(?:\n\n)[\s\S]+',r'\n\nREMOVED ATTACHMENT',i.as_string())
            content_type = i.get_content_type()
            filename = i.get_filename()
            charset = i.get_content_charset()
            md5 = hashlib.md5(i.get_payload(None,True)).hexdigest()
            sha256 = hashlib.sha256(i.get_payload(None,True)).hexdigest()
            replace_string = """
content_type=%(content_type)s
filename=%(filename)s
charset=%(charset)s
md5=%(md5)s
sha256=%(sha256)s
"""
            metadata = replace_string % dict(
                content_type=content_type, 
                filename=filename, 
                charset=charset,
                md5=md5,
                sha256=sha256,
            )
            new_payload += '\n' \
                + replace \
                + metadata
        new_payload += '\n--' + boundary
    new_payload += '--'
    message.set_payload(new_payload)
    return message.as_string()

In [5]:
def clean_move(input_dir=INPUT_DIR, output_dir=OUTPUT_DIR):
    for i in os.listdir(input_dir):
            f = os.path.join(input_dir, i)
            with open(f,'r',encoding='utf-8', errors='ignore') as raw_mail:
                mail = email.message_from_file(raw_mail)
                mail = find_message_attachment(mail)
                if mail:
                    mail = strip_attachments(mail)
                    with open(os.path.join(output_dir, i),'w') as out:
                        out.write(mail)
                else:
                    print(i)        

In [6]:
clean_move()

[Phish Alert] FW- Cathy V shared you a document via Dropbox[1].eml
[Phish Alert] FW- Compass review.eml
[Phish Alert] FW- Important Document.eml
[Phish Alert] FW- Engineered Demolition Project Lot#7694421[2].eml
[Phish Alert] FW- Redis Labs Enterprise Newsletter January, 2018[2].eml
[Phish Alert] FW- Burn Fat Zero Diet or Going to the Gym .eml
[Phish Alert] FW- Customer Service Representative jobs in Denver for Kirstin.eml
[Phish Alert] FW- SystemForward America, Inc..eml
FW- The Gift I Put In My Pocket.eml
[Phish Alert] FW- R.NICHOLS INVITED YOU TO VIEW[1].eml
[Phish Alert] FW- Happy Birthday!!.eml
[Phish Alert] FW- list - enquiry material[2].eml
[Phish Alert] FW- Hi Jolene.russell unlock pre-black friday card while it lasts.eml
RE- [Phish Alert] FW- Employee Accident AR-1779-1052 (External No. 1).eml
[Phish Alert] FW- R.NICHOLS INVITED YOU TO VIEW.eml
[Phish Alert] FW- Returned mail- see transcript for details.eml
[Phish Alert] FW- Change Three Marketing Elements in 2018.eml
[Phish A

In [7]:
len(os.listdir(INPUT_DIR))

4717

In [8]:
len(os.listdir(OUTPUT_DIR))

4681